# Dashboard Data Generator

This notebook is to generate data to power the dashboard.
The idea is to make csv extracts from the full dataset containing the necessary data for the dashboard

## Preliminaries

Please note the general set-up requirements contained in the repo readme and the _showcase_ notebook.
Remember that the database needs to be running locally for this workbook to work.

In [ ]:
#imports
import os  #to find the settings file(s)
import csv #to process the settings file(s)
import shutil #to copy the settings file (if needed)
from neo4j import GraphDatabase
import pandas as pd
from geopy.geocoders import Nominatim

In [ ]:
#get settings
settings_dir = os.path.join("..","settings")
personal_settings = os.path.join(settings_dir,"personal_settings.csv")
if not "personal_settings.csv" in os.listdir(settings_dir):
    default_settings = os.path.join(settings_dir,"default_settings.csv")
    shutil.copy(default_settings, personal_settings)
    print("Created new personal settings file, this probably needs to be edited before proceeding.")
with open(personal_settings, mode = 'r') as file:
    user_settings = {}
    for line in csv.DictReader(file):
        user_settings[line['setting']] = line['value']
db_uri = "bolt://localhost:" + str(user_settings['port_number'])

In [ ]:
#data paths
data_store = os.path.join("..","data","extracts","dutch_addresses")
data_dashboard = os.path.join("..","dashboard","data")

In [ ]:
db_connection = GraphDatabase.driver(db_uri, auth=(user_settings['username'],user_settings['password']))

In [ ]:
db_session = db_connection.session(database=user_settings['db_name'])

In [ ]:
#functions to add coordinates to an address
api_batch_size = 100
geolocator = Nominatim(user_agent="nl-application")  #initiate external tool to get coordinates from addresses

def add_coordinates(row):  #lambda function to add coordinates to a row in a dataframe
    temp_location = geolocator.geocode(row['address'])
    if(temp_location):
        row['longitude'] = temp_location.longitude
        row['latitude'] = temp_location.latitude
    row['api_check'] = True
    return row

## Create File Listing NL addresses by leak

In [ ]:
query = "MATCH (n:Address) WHERE n.country_codes CONTAINS 'NLD' RETURN n"
query_response = db_session.run(query)
addresses_nl = pd.DataFrame([dict(record.data()['n']) for record in query_response])

In [ ]:
addresses_nl = addresses_nl[['node_id','address','leak']]

In [ ]:
addresses_nl.head(5)

### Test geolocator

We use an external service to translate the addresses to coordinates so that they can be easily plotted on a map.

In [ ]:
location = geolocator.geocode(addresses_nl.iloc[0]['address'])
location

In [ ]:
location.longitude

In [ ]:
location.latitude

### Add Coordinates

The free online tools for finding coordinates from addresses have some limitations.
Firstly, there are limitations about how often you can make a request.
Secondly, if the address is not specified in the right format, the request can fail.
I suspect that both of these are playing a factor in why the coordinate requests are producing only limited results.
For this reason, we will do it in batches and the code here should _remember_ what coordinates have already been found.

The data structure we use here is the following. 
We take node_id (for easy indexing later), address and leak as our starting point.
In addition, we add longitude and latitude columns together with a an api check column and manual check column.
This latter column allows for modifying the address into a format that the API likes.
The api check column is to flag when we tried to find the address and failed.

In [ ]:
save_columns = ['node_id', 'address', 'leak', 'api_check', 'manual_address', 'longitude', 'latitude']

In [ ]:
#check if there are any saved addresses, and save new if not found
if len(os.listdir(data_store))==0:
    for column in save_columns:
        if column not in addresses_nl.columns:
            if column == 'api_check':
                addresses_nl[column] = False
            else:
                addresses_nl[column] = None
    addresses_nl.to_csv(os.path.join(data_store,"addresses_nl.csv"), index=False)
#load saved addresses
saved_addresses = pd.read_csv(os.path.join(data_store,"addresses_nl.csv"), index_col ='node_id')
saved_addresses.head(3)

In [ ]:
saved_addresses[saved_addresses['api_check']].shape[0]  #number of checked addresses

In [ ]:
saved_addresses[~saved_addresses['longitude'].isnull()].shape[0]  #number of found coordinates

In [ ]:
#create subset of addresses to be passed to the api
address_sample = saved_addresses[saved_addresses['api_check']==False].iloc[0:api_batch_size] #filter out already checked addresses and limit size
address_sample.head(5)

In [ ]:
#pass sample addresses to the api
address_sample = address_sample.apply(add_coordinates, axis=1)
address_sample.head(5)

In [ ]:
#merge back address sample to main dataset
saved_addresses.update(address_sample)

In [ ]:
saved_addresses[saved_addresses['api_check']].shape[0]  #number of checked addresses

In [ ]:
saved_addresses[~saved_addresses['longitude'].isnull()].shape[0]  #number of found coordinates

In [ ]:
#save data
saved_addresses.to_csv(os.path.join(data_store,"addresses_nl.csv"), index=True)

In [ ]:
#export data to dashboard
saved_addresses.to_csv(os.path.join(data_dashboard,"addresses_nl.csv"), index=True)

In [ ]:
#TODO: run on more addresses
#TODO: include more data columns in the addresses data set
#TODO: deal with addresses not found automatically